Development of the data pipeline for the raw measurements from Project Breathe

In [1]:
import pandas as pd
import src.data.helpers as dh
import src.data.breathe_data as breathe_data

In [14]:
path = (
    dh.get_path_to_main()
    + "DataFiles/BR/MeasurementData/Breathe_Spirometer_20231113.csv"
)
df = pd.read_csv(path)

In [15]:
print(df.columns)
print(df.shape)

Index(['UserId', 'EntityId', 'Timestamp', 'ClientTimestamp', 'IsDeleted',
       'CaptureType', 'CaptureDeviceMake', 'CaptureDeviceModel',
       'ThirdPartyId', 'PEF', 'PEFTime', 'FEV6', 'FEV1', 'FEV1PersonalBest',
       'FEV075', 'FEV1DivFEV6', 'FEV1DivFVC', 'FEF2575', 'FEV1Percent', 'EVol',
       'FVC', 'InvalidEntry', 'VolumeTimeCurve', 'FlowVolumeCurve',
       'IsCapturePrimary', 'CaptureSessionId'],
      dtype='object')
(104324, 26)


In [16]:
df.IsDeleted.unique()

array([False,  True])

In [17]:
tmp_size = df.shape[0]
df = df[~df.IsDeleted]
print(f"Removed {tmp_size - df.shape[0]} entries manually deleted on the app")

Removed 2565 entries manually deleted on the app


In [18]:
df.describe()

,EntityId,PEF,PEFTime,FEV6,FEV1,FEV1PersonalBest,FEV075,FEV1DivFEV6,FEV1DivFVC,FEF2575,FEV1Percent,EVol,FVC
count,1.017590e+05,87122.000000,4549.0,100680.000000,101759.000000,96319.000000,96319.000000,96319.000000,4163.000000,98277.000000,101176.000000,4549.0,7634.000000
mean,1.023662e+06,279.116710,0.0,3.029517,2.130394,3.262218,1.896338,0.681364,30.227024,1.631205,60.820797,0.0,1.116860
std,3.904265e+04,199.826929,0.0,1.068609,0.880368,0.979416,0.857896,0.176934,33.537712,1.169019,25.287741,0.0,1.790167
min,1.000001e+06,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,1.002048e+06,147.000000,0.0,2.340000,1.560000,2.900000,1.420000,0.620000,0.000000,0.900000,45.000000,0.0,0.000000
50%,1.008223e+06,264.000000,0.0,2.770000,1.910000,3.500000,1.710000,0.690000,0.000000,1.190000,59.000000,0.0,0.000000
75%,1.029546e+06,435.000000,0.0,3.650000,2.620000,3.900000,2.350000,0.800000,61.700001,2.130000,77.000000,0.0,1.980000
max,1.336051e+06,966.000000,0.0,9.830000,6.030000,5.300000,5.780000,1.000000,100.000000,8.680000,700.000000,0.0,7.130000


In [19]:
def count_na(series):
    return series.isna().sum()


df.agg(count_na)

UserId                     0
EntityId                   0
Timestamp                  0
ClientTimestamp            0
IsDeleted                  0
CaptureType                0
CaptureDeviceMake       5946
CaptureDeviceModel      5977
ThirdPartyId          101151
PEF                    14637
PEFTime                97210
FEV6                    1079
FEV1                       0
FEV1PersonalBest        5440
FEV075                  5440
FEV1DivFEV6             5440
FEV1DivFVC             97596
FEF2575                 3482
FEV1Percent              583
EVol                   97210
FVC                    94125
InvalidEntry               0
VolumeTimeCurve        99802
FlowVolumeCurve        99801
IsCapturePrimary           0
CaptureSessionId           0
dtype: int64

In [20]:
df["Datetime recorded"] = pd.to_datetime(df.Timestamp, utc=False)

In [35]:
df

,UserId,EntityId,Timestamp,ClientTimestamp,IsDeleted,CaptureType,CaptureDeviceMake,CaptureDeviceModel,ThirdPartyId,PEF,...,FEF2575,FEV1Percent,EVol,FVC,InvalidEntry,VolumeTimeCurve,FlowVolumeCurve,IsCapturePrimary,CaptureSessionId,Datetime recorded
0,60d96f45-ec56-470b-89ae-0085c9f073a7,1000001,2022-08-04 17:15:31.3786092,2022-07-29 16:19:05.1254750 +00:00,False,Bluetooth,Vitalograph,Lung Monitor BLE,NaN,326.0,...,2.53,88.0,NaN,NaN,False,NaN,NaN,True,00000000-0000-0000-0000-000000000000,2022-08-04 17:15:31.378609200
1,60d96f45-ec56-470b-89ae-0085c9f073a7,1000043,2022-08-04 17:15:31.3786092,2022-08-02 09:44:37.3587660 +00:00,False,Bluetooth,Vitalograph,Lung Monitor BLE,NaN,353.0,...,2.49,86.0,NaN,NaN,False,NaN,NaN,True,00000000-0000-0000-0000-000000000000,2022-08-04 17:15:31.378609200
2,60d96f45-ec56-470b-89ae-0085c9f073a7,1000044,2022-08-04 17:15:31.3786092,2022-08-02 09:45:39.0290770 +00:00,False,Bluetooth,Vitalograph,Lung Monitor BLE,NaN,351.0,...,2.34,87.0,NaN,NaN,False,NaN,NaN,True,00000000-0000-0000-0000-000000000000,2022-08-04 17:15:31.378609200
3,60d96f45-ec56-470b-89ae-0085c9f073a7,1000045,2022-08-04 17:15:31.3796091,2022-08-02 09:45:59.0738860 +00:00,False,Manual,NaN,NaN,NaN,NaN,...,NaN,57.0,NaN,NaN,False,NaN,NaN,True,00000000-0000-0000-0000-000000000000,2022-08-04 17:15:31.379609100
4,60d96f45-ec56-470b-89ae-0085c9f073a7,1000063,2022-08-02 15:25:26.4756283,2022-08-02 16:25:24.5166050 +00:00,False,Bluetooth,Vitalograph,Lung Monitor BLE,NaN,333.0,...,2.52,83.0,NaN,NaN,False,NaN,NaN,True,0a8aaa96-5a46-45a9-ab9b-82f2377fadb9,2022-08-02 15:25:26.475628300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104319,ac260677-b870-464e-bae8-ffff3309846f,1000635,2023-02-24 09:44:27.4674772,2023-02-24 09:44:25.7600840 +00:00,False,Bluetooth,Vitalograph,Lung Monitor BLE,NaN,267.0,...,0.96,39.0,NaN,NaN,True,NaN,NaN,True,f9a928f5-4e7e-42be-ad8f-585e96bfdf1e,2023-02-24 09:44:27.467477200
104320,ac260677-b870-464e-bae8-ffff3309846f,1000649,2023-05-23 12:41:49.0686095,2023-05-23 13:10:46.4215560 +01:00,False,Bluetooth,Vitalograph,Lung Monitor BLE,NaN,269.0,...,0.85,39.0,NaN,NaN,True,NaN,NaN,True,a4facf5e-6ac6-46c6-9eea-beac0cdd2a4b,2023-05-23 12:41:49.068609500
104321,ac260677-b870-464e-bae8-ffff3309846f,1000650,2023-05-23 12:41:49.1704273,2023-05-23 13:13:10.0901090 +01:00,False,Bluetooth,Vitalograph,Lung Monitor BLE,NaN,250.0,...,0.78,36.0,NaN,NaN,False,NaN,NaN,True,d704950d-1d19-4f72-a32d-b9873148c1d6,2023-05-23 12:41:49.170427300
104322,ac260677-b870-464e-bae8-ffff3309846f,1000651,2023-05-23 12:41:49.1764137,2023-05-23 13:15:39.3379010 +01:00,False,Bluetooth,Vitalograph,Lung Monitor BLE,NaN,269.0,...,0.86,41.0,NaN,NaN,True,NaN,NaN,True,8f631173-a58b-4f58-a510-2651363bcf3a,2023-05-23 12:41:49.176413700


In [21]:
cols_to_keep = [
    "UserId",
    "Datetime recorded",
    "FEV1",
    "FEF2575",
    "PEF",
    "InvalidEntry",
]
df1 = df[cols_to_keep]

In [22]:
df1.head()

,UserId,Datetime recorded,FEV1,FEF2575,PEF,InvalidEntry
0,60d96f45-ec56-470b-89ae-0085c9f073a7,2022-08-04 17:15:31.378609200,2.73,2.53,326.0,False
1,60d96f45-ec56-470b-89ae-0085c9f073a7,2022-08-04 17:15:31.378609200,2.68,2.49,353.0,False
2,60d96f45-ec56-470b-89ae-0085c9f073a7,2022-08-04 17:15:31.378609200,2.70,2.34,351.0,False
3,60d96f45-ec56-470b-89ae-0085c9f073a7,2022-08-04 17:15:31.379609100,2.88,NaN,NaN,False
4,60d96f45-ec56-470b-89ae-0085c9f073a7,2022-08-02 15:25:26.475628300,2.57,2.52,333.0,False


In [24]:
def get_BR_ID_to_partition_key_mapping():
    df = pd.read_excel(
        dh.get_path_to_main() + "ExcelFiles/BR/PredModInputData.xlsx",
        sheet_name="brPatient",
        usecols="A, AD",
    )
    # Set ID as string
    df.ID = df.ID.astype(str)
    return df


df_key_map = get_BR_ID_to_partition_key_mapping()

In [25]:
# Is mapping complete?
keys_intersect = set(df1.UserId).intersection(set(df_key_map.PartitionKey))
print(f"{len(keys_intersect)} intersecting keys")
keys_missing_in_map = set(df1.UserId) - set(df_key_map.PartitionKey)
print(f"{len(keys_missing_in_map)} keys from physdata are missing in mapping table")
keys_missing_in_physdata = set(df_key_map.PartitionKey) - set(df1.UserId)
print(
    f"{len(keys_missing_in_physdata)} keys from mapping table are missing in phys data"
)

239 intersecting keys
215 keys from physdata are missing in mapping table
18 keys from mapping table are missing in phys data


In [26]:
df2 = df1.merge(df_key_map, left_on="UserId", right_on="PartitionKey", how="inner")
df2.ID.nunique()

240

In [32]:
df2

,UserId,Datetime recorded,FEV1,FEF2575,PEF,InvalidEntry,ID,PartitionKey
0,18b5c897-2445-4390-bfbe-019b70c20fda,2022-08-04 17:15:02.545056800,2.44,1.18,0.0,False,183,18b5c897-2445-4390-bfbe-019b70c20fda
1,18b5c897-2445-4390-bfbe-019b70c20fda,2022-08-04 17:15:02.545056800,2.52,1.35,0.0,False,183,18b5c897-2445-4390-bfbe-019b70c20fda
2,18b5c897-2445-4390-bfbe-019b70c20fda,2022-08-04 17:15:02.546055300,2.52,1.35,0.0,False,183,18b5c897-2445-4390-bfbe-019b70c20fda
3,18b5c897-2445-4390-bfbe-019b70c20fda,2022-08-04 17:15:02.546055300,2.84,0.00,0.0,False,183,18b5c897-2445-4390-bfbe-019b70c20fda
4,18b5c897-2445-4390-bfbe-019b70c20fda,2022-08-04 17:15:02.546055300,2.82,1.53,0.0,False,183,18b5c897-2445-4390-bfbe-019b70c20fda
...,...,...,...,...,...,...,...,...
76714,c577a4f3-5dc7-4c07-9072-ff896c6b1316,2022-08-10 05:30:16.027219400,3.41,3.04,483.0,True,273,c577a4f3-5dc7-4c07-9072-ff896c6b1316
76715,c577a4f3-5dc7-4c07-9072-ff896c6b1316,2022-08-10 05:30:16.027219400,3.27,2.97,492.0,False,273,c577a4f3-5dc7-4c07-9072-ff896c6b1316
76716,c577a4f3-5dc7-4c07-9072-ff896c6b1316,2022-08-10 05:31:57.706153700,3.29,3.15,471.0,True,273,c577a4f3-5dc7-4c07-9072-ff896c6b1316
76717,c577a4f3-5dc7-4c07-9072-ff896c6b1316,2022-08-10 05:32:50.942244600,3.31,3.09,472.0,False,273,c577a4f3-5dc7-4c07-9072-ff896c6b1316


In [30]:
df1.shape

(101759, 6)

In [182]:
# Why 1 off error??

In [27]:
df3 = df2.drop(columns=["UserId", "PartitionKey"])

In [28]:
df3

,Datetime recorded,FEV1,FEF2575,PEF,InvalidEntry,ID
0,2022-08-04 17:15:02.545056800,2.44,1.18,0.0,False,183
1,2022-08-04 17:15:02.545056800,2.52,1.35,0.0,False,183
2,2022-08-04 17:15:02.546055300,2.52,1.35,0.0,False,183
3,2022-08-04 17:15:02.546055300,2.84,0.00,0.0,False,183
4,2022-08-04 17:15:02.546055300,2.82,1.53,0.0,False,183
...,...,...,...,...,...,...
76714,2022-08-10 05:30:16.027219400,3.41,3.04,483.0,True,273
76715,2022-08-10 05:30:16.027219400,3.27,2.97,492.0,False,273
76716,2022-08-10 05:31:57.706153700,3.29,3.15,471.0,True,273
76717,2022-08-10 05:32:50.942244600,3.31,3.09,472.0,False,273


In [185]:
# Understand invalid entry

df3.ID.value_counts()
df3["Date recorded"] = df3["Datetime recorded"].dt.date
df_for_ID = df3[df3.ID == "101"].reset_index()
df_for_ID.InvalidEntry.value_counts()

InvalidEntry
True     12275
False     3900
Name: count, dtype: int64

In [186]:
# Has invalid entry
df_for_ID[df_for_ID["Date recorded"] == df_for_ID["Date recorded"][0]]

,index,Datetime recorded,FEV1,FEF2575,PEF,InvalidEntry,ID,Date recorded
0,49269,2022-08-04 17:16:15.691332000,1.31,0.54,0.0,False,101,2022-08-04
1,49270,2022-08-04 17:16:15.692332100,1.31,0.57,0.0,False,101,2022-08-04
2,49271,2022-08-04 17:16:15.692332100,1.31,0.67,0.0,False,101,2022-08-04
3,49272,2022-08-04 17:16:15.692332100,1.30,0.69,0.0,False,101,2022-08-04
4,49273,2022-08-04 17:16:15.692332100,1.28,0.60,0.0,False,101,2022-08-04
...,...,...,...,...,...,...,...,...
5720,54989,2022-08-04 08:32:02.173501500,1.64,0.88,176.0,True,101,2022-08-04
5721,54990,2022-08-04 08:32:02.173501500,1.54,0.91,168.0,False,101,2022-08-04
5722,54991,2022-08-04 08:32:02.173501500,1.67,0.97,172.0,True,101,2022-08-04
5723,54992,2022-08-04 08:32:02.173501500,1.63,0.94,170.0,True,101,2022-08-04


# Data post processing

In [2]:
df = breathe_data.build_O2_FEV1_df()


*** Building O2 Saturation and FEV1 dataframe ***

*** Loading patients data ***
Warning - 4 NaN values in dataframe
The 4 NaN values belong to IDs ('322', '338', '344', '348') whose height are missing.
However, we don't correct for them as we don't have any measurement corresponding to those IDs for now.
Loaded 258 individuals

*** Loading measurements data ***
Dropping 1 entries with FEV1 = 6.0 for ID 330
* Checking for same day measurements *
* Checking for same day measurements *
Number of IDs:  243
Number of rows:  48976
Number of FEV1 recordings: 41791
Number of O2 Saturation recordings: 43930
Dropped 12231 entries with at least one NaN in subset ['O2 Saturation', 'FEV1']
This includes dropping 5046 entries with NaN O2 Saturation
This includes dropping 7185 entries with NaN FEV1
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 147.24696075623552
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 147.24696075623552
Warning for ID 202: FEV1 % Predicted shou

In [3]:
df_old = breathe_data.build_O2_FEV1_df(meas_file=1)


*** Building O2 Saturation and FEV1 dataframe ***

*** Loading patients data ***
Warning - 4 NaN values in dataframe
The 4 NaN values belong to IDs ('322', '338', '344', '348') whose height are missing.
However, we don't correct for them as we don't have any measurement corresponding to those IDs for now.
Loaded 258 individuals

*** Loading measurements data ***
Dropping 1 entries with FEV1 = 6.0 for ID 330
* Checking for same day measurements *
* Checking for same day measurements *
Number of IDs:  233
Number of rows:  26812
Number of FEV1 recordings: 23778
Number of O2 Saturation recordings: 23431
Dropped 6415 entries with at least one NaN in subset ['O2 Saturation', 'FEV1']
This includes dropping 3381 entries with NaN O2 Saturation
This includes dropping 3034 entries with NaN FEV1
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 147.24696075623552
Warning for ID 202: FEV1 % Predicted should be in 0-140%, got 147.24696075623552
Warning for ID 202: FEV1 % Predicted shoul

In [3]:
# df_old.to_excel(dh.get_path_to_main() + "ExcelFiles/BR/BR_O2_FEV1_FEF2575.xlsx", index=False)

# Processing with PEF

In [2]:
df = breathe_data.build_O2_FEV1_FEF2575_df()


*** Building O2Sat, FEV1, FEF2575 dataframe ***

*** Loading patients data ***
Warning - 4 NaN values in dataframe
The 4 NaN values belong to IDs ('322', '338', '344', '348') whose height are missing.
However, we don't correct for them as we don't have any measurement corresponding to those IDs for now.
Loaded 258 individuals

*** Loading measurements data ***
Dropping 1 entries with FEV1 = 6.0 for ID 330
* Checking for same day measurements *
* Checking for same day measurements *
* Checking for same day measurements *
* Checking for same day measurements *
Number of IDs:  243
Number of rows:  36341
Number of FEV1 recordings: 7773
Number of FEF2575 recordings: 7127
Number of PEF recordings: 6961
Number of O2 Saturation recordings: 35105
Dropped 30343 entries with at least one NaN in subset ['O2 Saturation', 'FEV1', 'FEF2575', 'PEF']
36341/36341 entries remain
This includes dropping 1236 entries with NaN O2 Saturation
This includes dropping 28568 entries with NaN FEV1
This includes dr

In [5]:
df.head()

,ID,Date Recorded,FEV1,O2 Saturation,FEF2575,PEF,ecFEV1,Age,Sex,Height,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy
0,101,2022-08-03,1.71,98.0,1.03,192.0,1.78,53,Male,173.0,3.610061,97.150104,49.306644,47.367618,100.874827
1,101,2022-08-04,1.78,98.0,1.32,266.0,1.78,53,Male,173.0,3.610061,97.150104,49.306644,49.306644,100.874827
2,101,2022-08-05,1.70,98.0,1.00,204.0,1.78,53,Male,173.0,3.610061,97.150104,49.306644,47.090615,100.874827
3,101,2022-08-06,1.71,98.0,1.03,181.0,1.71,53,Male,173.0,3.610061,97.150104,47.367618,47.367618,100.874827
4,101,2022-08-07,1.65,98.0,1.00,185.0,1.75,53,Male,173.0,3.610061,97.150104,48.475633,45.705597,100.874827


# Debugging breathe process

In [20]:
import datetime as datetime

In [21]:
import pytz

unaware = datetime.datetime(2011, 8, 15, 8, 15, 12, 0)
aware = datetime.datetime(2011, 8, 15, 8, 15, 12, 0, pytz.UTC)

In [29]:
b = pd.Series(
    [
        datetime.datetime(2011, 8, 15, 8, 15, 12, 0, pytz.UTC),
        datetime.datetime(2011, 8, 15, 8, 15, 12, 0, pytz.UTC),
    ]
)

In [28]:
type(b[0])

pandas._libs.tslibs.timestamps.Timestamp

In [31]:
b.dt.tz_localize(None)

0   2011-08-15 08:15:12
1   2011-08-15 08:15:12
dtype: datetime64[ns]

In [8]:
type(a[0])

str

In [18]:
b = pd.to_datetime(a, format='ISO8601', yearfirst=True)
type(b[0])

pandas._libs.tslibs.timestamps.Timestamp